# 🏋️ AI Search + Agent Service: Ejemplo Cursos🤸

Bienvenido a nuestro tutorial de AI Search + AI Agent, donde:

1. Crear un índice de Azure AI Search con algunos datos de ejemplo orientados al fitness  
2. Demostrar cómo conectar ese índice a un Agente mediante AzureAISearchTool  
3. Mostrar cómo consultar al Agente para obtener información sobre salud y fitness en un escenario divertido (¡con descargos de responsabilidad!)

## Requisitos Previos
1. Un recurso de Azure AI Search (anteriormente "Cognitive Search"), provisionado en su proyecto de Azure AI Foundry.

## Flujo General
Realizaremos lo siguiente:
1. Crear un índice de AI Search de forma programática con datos de ejemplo.
2. Subir documentos al índice.
3. Crear un Agente que haga referencia a nuestro nuevo índice usando AzureAISearchTool.
4. Ejecutar consultas para ver cómo se recupera la información del índice.


## 1. Create & Populate Azure AI Search Index

Crearemos un índice mínimo llamado `senaindex`, que contendrá algunos elementos de ejemplo. Asegúrate de configurar tus variables de entorno para `SEARCH_ENDPOINT` y `SEARCH_API_KEY`. Utilizaremos las clases de `azure.search.documents.indexes` para gestionar el esquema del índice. También subiremos algunos datos de ejemplo.


In [11]:
# Import required Azure libraries
import os
from azure.core.credentials import AzureKeyCredential  # For authentication
from azure.search.documents.indexes import SearchIndexClient  # For managing search indexes
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField  # Index schema components
from azure.search.documents import SearchClient  # For document operations (upload/search)
from azure.identity import DefaultAzureCredential  # For Azure authentication
from azure.ai.projects import AIProjectClient  # To access project resources
from azure.ai.projects.models import ConnectionType  # Enum for connection types
from pathlib import Path
from dotenv import load_dotenv

# First, initialize the AI Project client which gives us access to project resources
# This uses DefaultAzureCredential for authentication and the project connection string
# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '../.env')

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

# Get the Azure AI Search connection details from our project
# This includes endpoint URL and API key needed to access the search service
search_conn = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH, 
    include_credentials=True
)
if not search_conn:
    raise RuntimeError("❌ No default Azure AI Search connection found in your project.")

# Name of our search index - this is where our fitness data will be stored
index_name = "senaindex"

try:
    # Create a SearchIndexClient - this is used for managing the index itself (create/update/delete)
    credential = AzureKeyCredential(search_conn.key)
    index_client = SearchIndexClient(endpoint=search_conn.endpoint_url, credential=credential)
    print("✅ Created SearchIndexClient from project_client connection")
    
    # Create a SearchClient - this is used for document operations (upload/search/delete documents)
    # We'll use this later to add our fitness items to the index
    search_client = SearchClient(
        endpoint=search_conn.endpoint_url,
        index_name=index_name,
        credential=credential
    )
    print("✅ Created SearchClient for document operations")
    
except Exception as e:
    print(f"❌ Error creating search clients: {e}")

✅ Created SearchIndexClient from project_client connection
✅ Created SearchClient for document operations


**Define the index** schema with a `senaindex` key and a few fields to store product info.


In [ ]:
from azure.search.documents.indexes.models import (
    AzureOpenAIEmbeddingSkill,
    AzureOpenAIParameters,
    AzureOpenAIVectorizer,
    FieldMapping,
    HnswAlgorithmConfiguration,
    HnswParameters,
    IndexProjectionMode,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection,
    SearchIndexerDataSourceType,
    SearchIndexerIndexProjections,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    SearchIndexerSkillset,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SimpleField,
    SplitSkill,
    VectorSearch,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)

azure_openai_embedding_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]

def create_index():
    # Define the fields (columns) for our search index
    # Each field has specific attributes that control how it can be used in searches:
    fields = [
        # Primary key field - must be unique for each document
        SimpleField(name="Id", type=SearchFieldDataType.String, key=True),
        
        # Name field - SearchableField means we can do full-text search on it
        # filterable=True lets us filter results by name
        SearchableField(name="Name", type=SearchFieldDataType.String, filterable=True),
        
        # Category field - SearchableField for text search
        # filterable=True lets us filter by category
        # facetable=True enables category grouping in results
        SearchableField(name="Category", type=SearchFieldDataType.String, filterable=True, facetable=True),
        
        # Price field - SimpleField for numeric values
        # filterable=True enables price range filters
        # sortable=True lets us sort by price
        # facetable=True enables price range grouping
        SimpleField(name="Duration", type=SearchFieldDataType.Double, filterable=True, sortable=True, facetable=True),
        
        # Description field - SearchableField for full-text search on product descriptions
        SearchableField(name="Description", type=SearchFieldDataType.String)
    ]

    vector_search = VectorSearch(
                    algorithms=[
                        HnswAlgorithmConfiguration(name="algo", parameters=HnswParameters(metric=VectorSearchAlgorithmMetric.COSINE))
                    ],
                    vectorizers=[
                        AzureOpenAIVectorizer(
                            name="openai_vectorizer",
                            azure_open_ai_parameters=AzureOpenAIParameters(
                                resource_uri=azure_openai_embedding_endpoint,
                                deployment_id='text-embedding-ada-002',
                                model_name='text-embedding-ada-002'
                            )
                        )
                    ],
                    profiles=[
                        VectorSearchProfile(name="vp", algorithm_configuration_name="algo", vectorizer="openai_vectorizer")
                    ]
                )
    
    semantic_search=SemanticSearch(
        configurations=[
            SemanticConfiguration(
                name="default",
                prioritized_fields=SemanticPrioritizedFields(title_field=SemanticField(field_name="title"), content_fields=[SemanticField(field_name="description")])
            )
        ],
        default_configuration_name="default"
    )

    # Create an index definition with our fields
    index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_search=semantic_search)

    # Check if index already exists - if so, delete it to start fresh
    # This is useful during development but be careful in production!
    if index_name in [x.name for x in index_client.list_indexes()]:
        index_client.delete_index(index_name)
        print(f"🗑️ Deleted existing index: {index_name}")

    # Create the new index with our schema
    created = index_client.create_index(index)
    print(f"🎉 Created index: {created.name}")

# Execute the function to create our search index
create_index()

NameError: name 'AzureOpenAIParameters' is not defined

**Upload some sample documents** to `myfitnessindex`. We'll add a few items for demonstration.


In [3]:
def upload_docs():
    # Create a SearchClient to interact with our search index
    # This uses the connection details (endpoint, key) we configured earlier
    search_client = SearchClient(
        endpoint=search_conn.endpoint_url,
        index_name=index_name,
        credential=AzureKeyCredential(search_conn.key)
    )

    # Define sample documents that match our index schema
    # Each document must have:
    # - FitnessItemID (unique identifier)
    # - Name (searchable product name) 
    # - Category (searchable and facetable for filtering/grouping)
    # - Price (numeric field for sorting and filtering)
    # - Description (searchable product details)
    sample_docs = [
        {
            "Id": "1",
            "Name": "Curso Profesional de Data Science",
            "Category": "Professional Courses",
            "Duration": 40.0,
            "Description": "Un curso intensivo diseñado para profesionales que buscan profundizar en Data Science."
        },
        {
            "Id": "2",
            "Name": "Curso Profesional de Desarrollo Web",
            "Category": "Professional Courses",
            "Duration": 35.0,
            "Description": "Aprende tecnologías modernas de desarrollo web en un entorno profesional."
        },
        {
            "Id": "3",
            "Name": "Curso Profesional de Ciberseguridad",
            "Category": "Professional Courses",
            "Duration": 45.0,
            "Description": "Un curso completo para dominar la seguridad informática y evitar amenazas."
        },
        {
            "Id": "4",
            "Name": "Curso Profesional de Gestión de Proyectos",
            "Category": "Professional Courses",
            "Duration": 30.0,
            "Description": "Adquiere habilidades para gestionar proyectos de manera eficiente en ambientes corporativos."
        }
    ]

    # Upload all documents to the search index in a single batch operation
    # The search service will index these documents, making them searchable
    # based on the field types we defined in our index schema
    result = search_client.upload_documents(documents=sample_docs)
    print(f"🚀 Upload result: {result}")

# Call the function to upload the documents
upload_docs()
print("✅ Documents uploaded to search index")


🚀 Upload result: [<azure.search.documents._generated.models._models_py3.IndexingResult object at 0x000002298DD32490>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x000002298DD324D0>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x000002298DD32550>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x000002298DD325D0>]
✅ Documents uploaded to search index


### Verify the documents via a basic query

Hagamos una búsqueda rápida de cursos de **Ciberseguridad**.


In [4]:
# Let's verify our index by performing a basic search
# 1. First create a SearchClient using our connection details
#    - endpoint_url: The URL of our search service
#    - index_name: The name of the index we created earlier
#    - key: The admin key to authenticate our requests
search_client = SearchClient(
    endpoint=search_conn.endpoint_url,
    index_name=index_name,
    credential=AzureKeyCredential(search_conn.key)
)

# 2. Perform a simple search query:
#    - search_text="Strength": Look for documents containing "Strength"
#    - filter=None: No additional filtering
#    - top=10: Return up to 10 matching documents
results = search_client.search(search_text="Ciberseguridad", filter=None, top=10)

# 3. Print each matching document
print("🔍 Search results for 'Strength':")
print("-" * 50)
found_items = False
for doc in results:
    found_items = True
    # The doc is already a dictionary, no need for to_dict()
    print(f"Name: {doc['Name']}")
    print(f"Category: {doc['Category']}")
    print(f"Duration: {doc['Duration']:.2f} horas")
    print(f"Description: {doc['Description']}")
    print("-" * 50)

if not found_items:
    print("No matching items found.")

🔍 Search results for 'Strength':
--------------------------------------------------
Name: Curso Profesional de Ciberseguridad
Category: Professional Courses
Duration: 45.00 horas
Description: Un curso completo para dominar la seguridad informática y evitar amenazas.
--------------------------------------------------


Crearemos un nuevo agente y adjuntaremos un `AzureAISearchTool` referenciando **senaindex**.  

En tu entorno, necesitas:  
- `PROJECT_CONNECTION_STRING` - del resumen de tu proyecto AI Foundry  
- `MODEL_DEPLOYMENT_NAME` - del nombre del modelo desplegado  

Inicialicemos el `AIProjectClient` con `DefaultAzureCredential`.


In [5]:
# Import required libraries:
# - os: For accessing environment variables
# - DefaultAzureCredential: Azure's authentication mechanism
# - AIProjectClient: Main client for interacting with AI Projects
# - AzureAISearchTool & ConnectionType: Used to configure search capabilities
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool, ConnectionType

# Initialize the AI Project Client which we'll use to:
# 1. Connect to our Azure AI project
# 2. Create agents with search capabilities
# 3. Manage project resources
try:
    project_client = AIProjectClient.from_connection_string(
        # Use Azure's default authentication method
        credential=DefaultAzureCredential(),
        # Connect using the project connection string from environment variables
        conn_str=os.environ["PROJECT_CONNECTION_STRING"],
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"❌ Error initializing project client: {e}")

✅ Successfully initialized AIProjectClient


### Find (or create) the Azure AI Search connection in your Foundry project
We'll now use `project_client.connections.get_default(...)` to retrieve the default Azure AI Search connection, **including** credentials.


In [6]:
# Try to get the default Azure AI Search connection from our project
# - Azure AI Search (formerly Cognitive Search) is a cloud search service that helps 
#   us add search capabilities to our applications
# - The connection contains endpoint and credential information needed to access the search service
search_conn = project_client.connections.get_default(
    # Specify we want an Azure AI Search connection type
    connection_type=ConnectionType.AZURE_AI_SEARCH,
    # include_credentials=True means we'll get the full connection info including auth keys
    include_credentials=True
)

# Check if we found a connection
if not search_conn:
    print("❌ No default Azure AI Search connection found in your project.")
else:
    # If found, print the connection details
    # - The connection ID is a unique identifier for this connection in our project
    # - The endpoint URL is where our search service is hosted
    print(f"Found default Azure AI Search connection ID: {search_conn.id}")
    print(f"Index endpoint: {search_conn.endpoint_url}")

Found default Azure AI Search connection ID: /subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-ai-foundry-sena/providers/Microsoft.MachineLearningServices/workspaces/project-aytvwwktokp6w/connections/labaisearchsena
Index endpoint: https://lab-aisearch-sena.search.windows.net


### Create the Agent with `AzureAISearchTool`
We'll attach the tool, specifying the index name we created.


In [7]:
# Get the model deployment name from environment variables
# This is the Azure OpenAI model we'll use for our agent
model_name = "gpt-4o-2024-08-06"
agent = None

if search_conn:
    # Create an Azure AI Search tool that will allow our agent to search the fitness equipment index
    # - The tool needs the connection ID we got earlier to authenticate
    # - index_name specifies which search index to query (we created myfitnessindex earlier)
    ai_search_tool = AzureAISearchTool(
        index_connection_id=search_conn.id,
        index_name=index_name
    )

    # Create an AI agent that can understand natural language and search our index
    # - The agent uses our Azure OpenAI model for natural language understanding
    # - We give it instructions to act as a fitness shopping assistant
    # - We attach the search tool so it can look up products
    # - tool_resources provides the connection details the tool needs
    agent = project_client.agents.create_agent(
        model=model_name,
        name="sena-agent-search",
        instructions="""
        Eres VirtualSena con acceso a búsquedas en Azure Search,  un asesor virtual amigable especializado en recomendar cursos virtuales.
        **Siempre usa Azure Search para recomendar cusros**
        **No uses tu conocimiento previo.**
        Recuerda siempre a los usuarios: No soy un asesor académico oficial.
        Proporciona recomendaciones claras de cursos, explica brevemente cada uno y anima a los usuarios a explorar oportunidades de aprendizaje virtual.
        
        Siempre:
        1. Proporciona descargos de responsabilidad indicando que no eres un profesional del SENA.
        2. Incentiva la consulta profesional.
        3. Siempre usa Azure Search.
        4. Ofrece respuestas breves y útiles.
        """,
        tools=ai_search_tool.definitions,
        tool_resources=ai_search_tool.resources,
        headers={"x-ms-enable-preview": "true"},  # Enable preview features
    )
    print(f"🎉 Created agent, ID: {agent.id}")

🎉 Created agent, ID: asst_8wpUejydrIst0N0pJklY089a


## 3. Run a Conversation with the Agent

Abriremos un nuevo `thread`, publicaremos una pregunta y dejaremos que el agente busque en el índice los elementos relevantes.es: Abriremos un nuevo hilo, publicaremos una pregunta y dejaremos que el agente busque en el índice los elementos relevantes.

In [10]:
def run_agent_query(question: str):
    # Step 1: Create a new conversation thread
    # In Azure AI Agent service, conversations happen in threads, similar to chat conversations
    # Each thread can contain multiple back-and-forth messages
    thread = project_client.agents.create_thread()
    print(f"📝 Created thread, ID: {thread.id}")

    # Step 2: Add the user's question as a message in the thread
    # Messages have roles ("user" or "assistant") and content (the actual text)
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=question
    )
    print(f"💬 Created user message, ID: {message.id}")

    # Step 3: Create and start an agent run
    # This tells the agent to:
    # - Read the user's message
    # - Use its AI Search tool to find relevant products
    # - Generate a helpful response
    run = project_client.agents.create_and_process_run(
        thread_id=thread.id,
        assistant_id=agent.id
    )
    print(f"🤖 Agent run status: {run.status}")

    # Check for any errors during the agent's processing
    if run.last_error:
        print("⚠️ Run error:", run.last_error)

    # Step 4: Get the agent's response
    # Retrieve all messages and find the most recent assistant response
    # The response might contain multiple content blocks (text, images, etc.)
    msg_list = project_client.agents.list_messages(thread_id=thread.id)
    for m in reversed(msg_list.data):
        if m.role == "assistant" and m.content:
            print("\nAssistant says:")
            for c in m.content:
                if hasattr(c, "text"):
                    print(c.text.value)
            break

# Try out our agent with two example queries:
# 1. A general question about strength training equipment
# 2. A specific request for cardio equipment with a price constraint
if agent:
    run_agent_query("Cuales cursos tienes en cibersecurity?")

📝 Created thread, ID: thread_EH9EJggFie2kS0aqHOqifNSZ
💬 Created user message, ID: msg_BPkj1SLDdVodLB3Y7Lv5l3FD
🤖 Agent run status: RunStatus.FAILED
⚠️ Run error: {'code': 'tool_user_error', 'message': 'Error: invalid_search_request; Invalid configuration for Azure AI Search index. Query type vector_semantic_hybrid requires a vector field with integrated vectorizer, but none was found'}
